# Лабораторная работа №1. Курс: прикладная математика. Авторы: Ярослав Ведерников и Никита Хауров. Группа: М33011.

## Минимизация линейной формы с помощью Симплекс-метода

## Задачи

- Реализуйте возможность ввода данных из файла в формате JSON. Рекомендуемая структура JSON указана ниже
- При необходимости добавьте балансирующие переменные для перехода от общей постановки к канонической форме задачи линейного программирования
- Реализуйте симплекс-метод для решения задачи
- Предусмотрите, что задача как может не иметь решений вообще, так и иметь
бесконечное количество решений

In [32]:
import json
import copy

### Вспомогательный класс, выполняющий задачу парсинга JSON

In [33]:
class JsonTask:
    def __init__(self, json_file):
        json_content = json.loads(json_file)
        self.function = json_content["f"]
        self.goal = json_content["goal"]
        self.constraints = [phrase for phrase in json_content["constraints"]]

### Решение с помощью симплекс метода представлено в классе

In [34]:
class Simplex:
    def __init__(self, task: JsonTask):
        if task.goal == 'min': self.function = task.function
        else: self.function = [-num for num in task.function]
        self.goal = task.goal

        self.matrix = []  # матрица симплекс метода
        for limit in task.constraints:
            if limit["type"] == 'eq':  # уравнение переводится в систему из двух нестрогих неравенств
                self.matrix.append(copy.deepcopy(limit["coefs"]) + [limit["b"]])
                self.matrix.append([-c for c in copy.deepcopy(limit["coefs"])] + [-limit["b"]])
            elif limit["type"] == 'lte': self.matrix.append(limit["coefs"] + [limit["b"]])
            elif limit["type"] == 'gte': self.matrix.append([-num for num in limit["coefs"]] + [-limit["b"]])
            else: print('Ошибка: неизвестный знак в неравенстве.')
        self.matrix.append([-num for num in self.function] + [0])  # добавляется последняя строка с целевой функцией
        self.variables_in_function = list(range(1, len(self.function) + 1))  # индексы переменных, входящих в функцию
        self.variables_in_basis = list(range(len(self.function) + 1, len(self.function) + len(self.matrix)))  # индексы переменных, входящих в базис


    def jordan_steps(self, crucial_row, crucial_column):
        matrix_before = copy.deepcopy(self.matrix)  # сохраняем матрицу до шага Жордана
        crucial_element = matrix_before[crucial_row][crucial_column]
        self.jordan_step_1(crucial_row, crucial_column)
        self.jordan_step_2(crucial_element, crucial_row, crucial_column, matrix_before)
        self.jordan_step_3(crucial_element, crucial_row, crucial_column, matrix_before)
        self.jordan_step_4(crucial_element, crucial_row, crucial_column, matrix_before)
        # чтобы иметь возможность полностью восстановить всё решение, отслеживаются индексы переменных в базисе и в функции
        excluded_var_from_basis = self.variables_in_basis.pop(crucial_row)
        excluded_var_from_func = self.variables_in_function.pop(crucial_column)
        self.variables_in_basis.insert(crucial_row, excluded_var_from_func)
        self.variables_in_function.insert(crucial_column, excluded_var_from_basis)


    def jordan_step_1(self, crucial_row, crucial_column):  # считается разрешающий элемент
        self.matrix[crucial_row][crucial_column] = 1 / self.matrix[crucial_row][crucial_column]


    def jordan_step_2(self, crucial_element, crucial_row, crucial_column, matrix_before):  # считаются элементы разрешающего столбца
        for row in (r for r in range(len(self.matrix)) if r != crucial_row):
            self.matrix[row][crucial_column] = -matrix_before[row][crucial_column] / crucial_element


    def jordan_step_3(self, crucial_element, crucial_row, crucial_column, matrix_before):  # считаются элементы разрешающей строки
        for col in (c for c in range(len(self.matrix[crucial_row])) if c != crucial_column):
            self.matrix[crucial_row][col] = matrix_before[crucial_row][col] / crucial_element


    def jordan_step_4(self, crucial_element, crucial_row, crucial_column, matrix_before):  # считаются все остальные элементы
        for row in (r for r in range(len(self.matrix)) if r != crucial_row):
            for col in (c for c in range(len(self.matrix[row])) if c != crucial_column):
                self.matrix[row][col] = matrix_before[row][col] - (matrix_before[crucial_row][col] * matrix_before[row][crucial_column] / crucial_element)


    def solve_task(self):
        self.base_plan()  # поиск опорного плана
        self.best_plan()  # поиск оптимального плана
        return self.get_correct_answer()
    

    def get_correct_answer(self):
        if (self.goal == "min"):
            return (self.matrix[-1][-1], self.variables_in_function, self.variables_in_basis, [-i for i in self.matrix[-1][:-1]], [row[-1] for row in self.matrix[:-1]])
        return (-self.matrix[-1][-1], self.variables_in_function, self.variables_in_basis, self.matrix[-1][:-1], [row[-1] for row in self.matrix[:-1]])


    def base_plan(self):
        negative_b = self.find_negative_b()
        while negative_b is not None:  # пока есть отрицательные коэффициенты в столбце свободных членов, делаем Жорданов шаг
            crucial_row, crucial_column = self.find_crucial_indices(negative_b)
            self.jordan_steps(crucial_row, crucial_column)
            negative_b = self.find_negative_b()


    def find_negative_b(self):
        for row in range(len(self.matrix) - 1):
            if self.matrix[row][-1] < 0: return row
        return None


    def find_crucial_indices(self, index_of_negative):  # ищутся индексы разрешающего элемента
        crucial_column = self.find_crucial_col(index_of_negative)
        if crucial_column is None: raise Exception('Ошибка: система ограничений несовместна.')
        return (self.find_crucial_row(crucial_column), crucial_column)


    def find_crucial_col(self, index_of_negative):
        for col in range(len(self.matrix[index_of_negative]) - 1):
            if self.matrix[index_of_negative][col] < 0: return col
        return None


    def find_crucial_row(self, crucial_column):
        min_value = float('inf')
        for row in range(len(self.matrix) - 1):
            try: val = self.matrix[row][-1] / self.matrix[row][crucial_column]
            except ZeroDivisionError: continue

            if val > 0 and val < min_value:
                min_value = val
                crucial_row = row
        return crucial_row


    def best_plan(self):
        crucial_column = self.find_best_plan_crucial_column()
        while crucial_column is not None:  # пока есть неотрицательные коэффициенты в строке целевой функции, делаем Жорданов шаг
            crucial_row = self.find_best_plan_crucial_row(crucial_column)
            if (crucial_row is None): raise Exception('Ошибка: функция не ограничена.')
            self.jordan_steps(crucial_row, crucial_column)
            crucial_column = self.find_best_plan_crucial_column()


    def find_best_plan_crucial_column(self):
        for col in range(len(self.matrix[-1]) - 1):
            if self.matrix[-1][col] > 0: return col
        return None


    def find_best_plan_crucial_row(self, crucial_column):
        min_value = float('inf')
        for row in range(len(self.matrix) - 1):
            try: val = self.matrix[row][-1] / self.matrix[row][crucial_column]
            except ZeroDivisionError: continue

            if self.matrix[row][crucial_column] > 0 and val < min_value:
                min_value = val
                crucial_row = row
        if min_value != float('inf'): return crucial_row
        return None


### Задача из файла с лабой

In [35]:
json_file_1 = '{"f": [0, 0, 0, 1, -1],"goal": "min","constraints": [{"coefs": [1, 0, 0, 1, -2], "type": "eq", "b": 1},{"coefs": [0, 1, 0, -2, 1], "type": "eq", "b": 2},{"coefs": [0, 0, 1, 3, 1], "type": "eq", "b": 3}]}'
answer, function_vars, variables_in_basis, function_coefs, b_column = Simplex(JsonTask(json_file_1)).solve_task()
print(f"Ответ: {answer}\nИндексы переменных базиса: {variables_in_basis}\nИндексы переменных функции: {function_vars}"
      f"\nКоэффициенты в целевой функции: {function_coefs}\nСтолбец свободных членов {b_column}")

Ответ: -2.2
Индексы переменных базиса: [1, 7, 5, 9, 4, 11]
Индексы переменных функции: [6, 8, 10, 3, 2]
Коэффициенты в целевой функции: [-0.0, 0.8, 0.2, 0.2, 0.8]
Столбец свободных членов [5.6, 0.0, 2.4, 0.0, 0.2, 0.0]


### Задача 1 из файла с тестами

In [36]:
test_2 = '{"f": [-6, -1, -4, 5],"goal": "min","constraints": [{"coefs": [3, 1, -1, 1], "type": "eq", "b": 4},{"coefs": [5, 1, 1, -1], "type": "eq", "b": 4}]}'
answer, function_vars, variables_in_basis, function_coefs, b_column = Simplex(JsonTask(test_2)).solve_task()
print(f"Ответ: {answer}\nИндексы переменных базиса: {variables_in_basis}\nИндексы переменных функции: {function_vars}"
      f"\nКоэффициенты в целевой функции: {function_coefs}\nСтолбец свободных членов {b_column}")

Ответ: -4.0
Индексы переменных базиса: [2, 5, 3, 8]
Индексы переменных функции: [7, 6, 1, 4]
Коэффициенты в целевой функции: [2.5, 1.5, 2.0, 1.0]
Столбец свободных членов [3.9999999999999996, 2.220446049250313e-16, 0.0, 0.0]


### Задача 2 из файла с тестами

In [37]:
test_2 = '{"f": [-1, -2, -3, 1],"goal": "min","constraints": [{"coefs": [1, -3, -1, -2], "type": "eq", "b": -4},{"coefs": [1, -1, 1, 0], "type": "eq", "b": 0}]}'
answer, function_vars, variables_in_basis, function_coefs, b_column = Simplex(JsonTask(test_2)).solve_task()
print(f"Ответ: {answer}\nИндексы переменных базиса: {variables_in_basis}\nИндексы переменных функции: {function_vars}"
      f"\nКоэффициенты в целевой функции: {function_coefs}\nСтолбец свободных членов {b_column}")

Ответ: -5.999999999999999
Индексы переменных базиса: [2, 5, 1, 8]
Индексы переменных функции: [7, 6, 3, 4]
Коэффициенты в целевой функции: [2.4999999999999996, 1.4999999999999998, 0.9999999999999991, 3.999999999999999]
Столбец свободных членов [1.9999999999999998, 0.0, 1.9999999999999998, 0.0]


### Задача 3 из файла с тестами

In [38]:
test_3 = '{"f": [-1, -2, -1, 3, -1],"goal": "min","constraints": [{"coefs": [1, 1, 0, 2, 1], "type": "eq", "b": 5},{"coefs": [1, 1, 1, 3, 2], "type": "eq", "b": 9}, {"coefs": [0, 1, 1, 2, 1], "type": "gte", "b": 6}]}'
answer, function_vars, variables_in_basis, function_coefs, b_column = Simplex(JsonTask(test_3)).solve_task()
print(f"Ответ: {answer}\nИндексы переменных базиса: {variables_in_basis}\nИндексы переменных функции: {function_vars}"
      f"\nКоэффициенты в целевой функции: {function_coefs}\nСтолбец свободных членов {b_column}")

Ответ: -14.0
Индексы переменных базиса: [10, 2, 3, 9, 7]
Индексы переменных функции: [1, 6, 8, 4, 5]
Коэффициенты в целевой функции: [1.0, 1.0, 1.0, 8.0, 2.0]
Столбец свободных членов [3.0, 5.0, 4.0, 0.0, 0.0]


## Вывод

Написанный симплекс-метод успешно решает задачи максимизации и минимизации линейных форм.